In [1]:
%matplotlib widget

from experiments import VelocityVSATPADPRatio, SC2RVSDiscSpiral

# Comparison between SC/2R and Disc-Spiral models

In [ ]:
sc2r_vs_disc_spiral = SC2RVSDiscSpiral()

# Relation between ATP/ADP concentrations ratio and velocity

In [2]:
velocity_vs_atp_adp_ratio = VelocityVSATPADPRatio()

No such comm: 97468d89f57c4deb9c2e05be142a74c3
